## IMPORTING LIBRARIES 

In [44]:
import pandas as pd
import json
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler  # Added MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential, load_model

## DATA PREPROCESSING

In [45]:
def preprocess_data(data_file, scaler_type, predict_feature):
    """
    Function to preprocess data by scaling the predict_feature column using the specified scaler type.

    Parameters:
    data_file (str): Path to the CSV file containing the data.
    scaler_type (str): Type of scaler to use. Either 'StandardScaler' or 'MinMaxScaler'.

    Returns:
    scaled_data (numpy.ndarray): The scaled data.
    scaler: The scaler object used for scaling the data.
    """
    import pandas as pd
    from sklearn.preprocessing import StandardScaler, MinMaxScaler

    # Read the data
    training_set = pd.read_csv(data_file)

    # Choose the scaler
    if scaler_type == 'StandardScaler':
        scaler = StandardScaler()
    elif scaler_type == 'MinMaxScaler':
        scaler = MinMaxScaler()
    else:
        raise ValueError(f"Invalid scaler type: {scaler_type}")

    # Scale the data
    scaled_data = scaler.fit_transform(training_set[predict_feature].values.reshape(-1, 1))

    return scaled_data, scaler


In [46]:
def split_into_sequences(data, seq_len):
    """
    Splits the data into sequences of a specified length.

    Parameters:
    data (numpy.ndarray): The input data.
    seq_len (int): The length of each sequence.

    Returns:
    numpy.ndarray: Array of sequences.
    """
    n_seq = len(data) - seq_len + 1
    return np.array([data[i:(i+seq_len)] for i in range(n_seq)])

def get_train_test_sets(data, seq_len, train_frac):
    """
    Splits the data into training and testing sets.

    Parameters:
    data (numpy.ndarray): The input data.
    seq_len (int): The length of each sequence.
    train_frac (float): The fraction of data to use for training.

    Returns:
    tuple: A tuple containing four numpy arrays: x_train, y_train, x_test, and y_test.
    """
    sequences = split_into_sequences(data, seq_len)
    n_train = int(sequences.shape[0] * train_frac)
    x_train = sequences[:n_train, :-1, :]
    y_train = sequences[:n_train, -1, :]
    x_test = sequences[n_train:, :-1, :]
    y_test = sequences[n_train:, -1, :]
    return x_train, y_train, x_test, y_test


## BUILDING MODEL AND TRAINING

In [47]:
def build_model(input_size, hidden_size, num_layers, num_classes):
    """
    Builds a Bidirectional LSTM model for sequence prediction.

    Parameters:
    input_size (int): The number of features in the input data.
    hidden_size (int): The number of units in the LSTM layer.
    num_layers (int): The number of LSTM layers.
    num_classes (int): The number of output classes.

    Returns:
    keras.models.Sequential: Compiled Bidirectional LSTM model.
    """

    # Create a Sequential model
    model = Sequential()

    # Add a Bidirectional LSTM layer
    model.add(Bidirectional(
        LSTM(hidden_size, input_shape=(None, input_size), return_sequences=False)
    ))

    # Add a Dense layer for output
    model.add(Dense(num_classes))

    # Compile the model
    model.compile(loss='mean_squared_error', optimizer='adam')

    return model


In [48]:
def train_model(model, x_train, y_train, num_epochs, patience, random_seed=None):
    """
    Trains a given model using the provided training data.

    Parameters:
    model (keras.models.Sequential): The model to be trained.
    x_train (numpy.ndarray): The input training data.
    y_train (numpy.ndarray): The target training data.
    num_epochs (int): The number of epochs for training.
    patience (int): The number of epochs with no improvement after which training will be stopped.
    random_seed (int): Random seed for reproducibility. If None, randomness is not controlled.

    Returns:
    keras.models.Sequential: The trained model.
    """
    if random_seed is not None:
        np.random.seed(random_seed)
        tf.random.set_seed(random_seed)

    best_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        history = model.fit(x_train, y_train, epochs=1, verbose=0)
        loss = history.history['loss'][0]

        if epoch % 25 == 0:
            print(f"Step: {epoch+1}/{num_epochs}, Loss: {loss:.6f}")

        if loss < best_loss:
            best_loss = loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early training stop. No improvement in loss for the last {patience} steps.")
            break
    return model

## GENERATING FORECASTS

In [49]:
def forecast_with_model(model, data_file, seq_len, scaler, predict_feature):
    """
    Generates forecasts using the provided model.

    Parameters:
    model (keras.models.Sequential): The trained model used for forecasting.
    data_file (str): The path to the CSV file containing the data.
    seq_len (int): The length of input sequences used for prediction.
    scaler: The scaler object used for scaling the input data.

    Returns:
    tuple: A tuple containing two lists: actuals and forecasts.
        - actuals (list): A list of actual values for each forecasting scenario.
        - forecasts (list): A list of forecasted values for each forecasting scenario.
    """

    # Read the data from the CSV file
    training_set1 = pd.read_csv(data_file)
    last_column_df = training_set1[[predict_feature]].copy()
    
    # Initialize lists to store forecasts and actuals
    forecasts = []
    actuals = []
    
    # Define percentages for splitting the data
    percentages = [
        0.9975, 0.995, 0.9925, 0.99, 0.9875, 0.985, 0.9825, 0.98, 0.9775, 0.975, 
        0.9725, 0.97, 0.965, 0.9625, 0.96, 0.9575, 0.955, 0.9525, 0.95, 0.9675
    ]

    # Iterate over each percentage
    for percentage in percentages:
        actual_data_01 = int(len(last_column_df) * percentage)
        actual_data_02 = int(len(last_column_df) * (1 - percentage))
        dataset_01 = last_column_df.iloc[:actual_data_01]
        dataset_02 = last_column_df.iloc[actual_data_01:]
        input_values = dataset_01.tail(seq_len-1)
        scaled_input_values = scaler.transform(input_values)
        x_pred = scaled_input_values.reshape(1, (seq_len-1), 1)
        input_values = input_values.values.flatten()
        forecast_values = []

        # Generate forecasts for the next 48 steps
        for _ in range(48):
            scaled_input_values = scaler.transform(input_values.reshape(-1, 1))
            x_pred = scaled_input_values[-(seq_len-1):].reshape(1, (seq_len-1), 1)
            predicted_value = model.predict(x_pred)
            predicted_value = scaler.inverse_transform(predicted_value)
            forecast_values.append(predicted_value[0][0])
            input_values = np.append(input_values, predicted_value)
        
        # Get actual values for comparison
        actual_values = dataset_02.head(48).values.flatten()
        forecasts.append(forecast_values)
        actuals.append(actual_values)
    
    return actuals, forecasts


## EVALUATING FORECASTS

In [50]:
def evaluate_forecast(actual_values, forecast_values):
    """
    Evaluates forecast accuracy using various error metrics.

    Parameters:
    actual_values (list of arrays): The actual values.
    forecast_values (list of arrays): The forecasted values.

    Returns:
    tuple: A tuple containing four average error metrics:
        - Mean Absolute Error (MAE)
        - Mean Squared Error (MSE)
        - Root Mean Squared Error (RMSE)
        - Mean Absolute Percentage Error (MAPE)
    """
    import numpy as np
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    from math import sqrt

    mae_values = []
    mse_values = []
    rmse_values = []
    mape_values = []

    # Calculate error metrics for each forecasting scenario
    for actual, forecast in zip(actual_values, forecast_values):
        mae = mean_absolute_error(actual, forecast)
        mse = mean_squared_error(actual, forecast)
        rmse = sqrt(mse)

        def mean_absolute_percentage_error(y_true, y_pred):
            return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

        mape = mean_absolute_percentage_error(actual, forecast)

        mae_values.append(mae)
        mse_values.append(mse)
        rmse_values.append(rmse)
        mape_values.append(mape)

    # Calculate average error metrics
    mae_avg = np.mean(mae_values)
    mse_avg = np.mean(mse_values)
    rmse_avg = np.mean(rmse_values)
    mape_avg = np.mean(mape_values)

    return mae_avg, mse_avg, rmse_avg, mape_avg


In [51]:
def calculate_fluctuations(data_file, time_column, predict_feature):
    """
    Calculates fluctuations in the predict_feature column of a time series dataset.

    Parameters:
    data_file (str): The path to the CSV file containing the time series data.

    Returns:
    tuple: A tuple containing two values:
        - total_std_dev (float): The standard deviation of predict_feature for the entire dataset.
        - total_iqr (float): The interquartile range (IQR) for the entire dataset.
    """
    import pandas as pd

    # Load the time series data
    time_series = pd.read_csv(data_file)[[time_column, predict_feature]]

    # Calculate the standard deviation of predict_feature for the entire dataset
    total_std_dev = time_series[predict_feature].std()

    # Calculate the interquartile range (IQR) for the entire dataset
    total_Q1 = time_series[predict_feature].quantile(0.25)
    total_Q3 = time_series[predict_feature].quantile(0.75)
    total_iqr = total_Q3 - total_Q1
    
    return total_std_dev, total_iqr


## MAIN FUNCTION

In [52]:
import pandas as pd
import numpy as np

def main(input_file, predict_feature, time_column):
    """
    Runs the main procedure for forecasting and evaluating models.

    This function iterates over different combinations of sequence lengths and scaler types
    to train, forecast, and evaluate models. It then identifies the best performing model
    based on Mean Squared Error (MSE) and prints the corresponding parameters.

    Parameters:
        input_file (str): Path to the input CSV file.
        predict_feature (str): Name of the column containing the feature to predict.
        time_column (str): Name of the column containing the time information.

    Returns:
        dict: Dictionary containing the best parameters (Sequence Length, Scaler Type, Best MSE).
    """
    results_data = []
    scaler_types = ['StandardScaler', 'MinMaxScaler']
    best_mse = np.inf
    best_seq_len = None
    best_scaler_type = None
    
    # Load the entire dataset
    full_dataset = pd.read_csv(input_file)

    # Calculate the index corresponding to the first 95% of the data
    n_rows = int(0.95 * len(full_dataset))

    # Get the first 95% of the data
    first_95_percent = full_dataset.head(n_rows)

    # Optionally, you can save this subset to a new CSV file
    output_file = input_file.replace('.csv', '_95.csv')
    first_95_percent.to_csv(output_file, index=False)

    for scaler_type in scaler_types:
        # Ensure calculate_fluctuations returns std_dev, iqr, mad
        tot_std_dev, tot_iqr = calculate_fluctuations(input_file, time_column, predict_feature)
        #print(std_dev,iqr)
        data, scaler = preprocess_data(output_file, scaler_type, predict_feature)

        seq_len = 12
        while seq_len <= 60:  # Adjust the maximum sequence length based on your requirements
            x_train, y_train, _, _ = get_train_test_sets(data, seq_len, train_frac=0.95)
            model = build_model(input_size=1, hidden_size=2, num_layers=1, num_classes=1)
            model = train_model(model, x_train, y_train, num_epochs=2000, patience=30)

            actuals, forecasts = forecast_with_model(model, input_file, seq_len, scaler, predict_feature)
            mae, mse, rmse, mape = evaluate_forecast(actuals, forecasts)
            
            # Check if the current MSE is the best so far
            if mse < best_mse:
                best_mse = mse
                best_seq_len = seq_len
                best_scaler_type = scaler_type
            
            # Collect data for the results
            results_data.append({
                'Scaler Type': scaler_type,
                'Sequence Length': seq_len,
                'Tot_Std Dev': tot_std_dev,
                'Tot_IQR': tot_iqr,
                'MAE': mae,
                'MSE': mse,
                'RMSE': rmse,
                'MAPE': mape
            })
            seq_len += 12

    # Save the results to a DataFrame
    results_df = pd.DataFrame(results_data)

    # Print the best parameters
    best_params = {
        'Sequence Length': best_seq_len,
        'Scaler Type': best_scaler_type,
        'Best MSE': best_mse
    }
    print("Best parameters:")
    print(best_params)

    return best_params


Step: 1/20, Loss: 0.397815


C:\Users\soham\anaconda3\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - ETA: 0s

C:\Users\soham\anaconda3\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - ETA: 0s

...

1/1 [==============================] - 0s 34ms/step


C:\Users\soham\anaconda3\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - ETA: 0s

C:\Users\soham\anaconda3\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - 0s 31ms/step
Best parameters:
{'Sequence Length': 3, 'Scaler Type': 'StandardScaler', 'Best MSE': 69.15315886708046}


In [53]:
# Example usage:
input_file = 'AT_combined.csv'
predict_feature = "Low Carbon Percentage"
time_column = "Datetime (UTC)"
best_params = main(input_file, predict_feature, time_column)

Step: 1/20, Loss: 0.332976


C:\Users\soham\anaconda3\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - 0s 20ms/step


C:\Users\soham\anaconda3\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - ETA: 0s

...

1/1 [==============================] - 0s 30ms/step


C:\Users\soham\anaconda3\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - 0s 19ms/step


C:\Users\soham\anaconda3\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - 0s 19ms/step
Best parameters:
{'Sequence Length': 3, 'Scaler Type': 'StandardScaler', 'Best MSE': 74.79563704806971}
